In [164]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import more_itertools as mit
from scipy import interpolate
from math import pow

In [151]:
def GetPrices(tickerPath):
    data = pandas.read_csv(tickerPath)
    return np.array(data["close"])

def GetDates(tickerPath):
    data = pandas.read_csv(tickerPath)
    return np.array(data["date"])

In [149]:
def TrendDuration(prices):
    returns = np.diff(closePrices)
    trendSigns = np.sign(returns).tolist()
    fixedTrendSigns = [x for x in trendSigns if x != 0.0]
    trendStartPoints = np.insert(np.where(np.diff(fixedTrendSigns))[0], 0, -1)
    trendDuration = np.diff(trendStartPoints)
    fixForGeometricModel = trendDuration - 1
    return fixForGeometricModel.tolist()

def ModelP(j):
    modelP = 0.5
    for i in range(j):
        modelP /= 2.0
    return modelP

def PEstimator(trendDurations, j):
    return trendDurations.count(j)/len(testData)

def Z(trendDurations, j):
    n = len(trendDurations)
    sjSum = 0.0
    tjSum = 0.0
    
    for i in range(j+1):
        sjSum += trendDurations.count(i)
        tjSum += n*ModelP(i)
    
        
    return sjSum - tjSum

def GeometricCramerVonMisesTest(trendDurations):
    testScore = 0.0
    n = len(trendDurations)
    m = max(trendDurations)
    for j in range(m+1):
        testScore += pow(Z(trendDurations, j), 2) * ModelP(j)
        
    return testScore / n

pValData = pandas.read_csv("testScoreToPValuePts.csv")
[testValue, pValue] = np.transpose(np.array(pValData)).tolist()
pValueFunction = interpolate.interp1d(testValue, pValue)

def GeometricCramerVonMisesPValue(trendDurations):
    testStatistic = GeometricCramerVonMisesTest(trendDurations)
    return float(pValueFunction(testStatistic))

In [152]:
dates = GetDates("SP500/AAPL.csv")

In [147]:
closePrices = GetPrices("SP500/AAPL.csv")

In [156]:
datedPrices = np.transpose([dates,closePrices])

In [199]:
def IsNotNone(dataPoint):
    return not dataPoint is None

partitions = list(mit.windowed(datedPrices, n = 10, step = 10))
partitions = [list(filter(IsNotNone, p)) for p in partitions]

In [201]:
partitions[1]

[array(['2007-08-20', 4.364999771118164], dtype=object),
 array(['2007-08-21', 4.556070804595947], dtype=object),
 array(['2007-08-22', 4.732500076293945], dtype=object),
 array(['2007-08-23', 4.681070804595947], dtype=object),
 array(['2007-08-24', 4.83214282989502], dtype=object),
 array(['2007-08-27', 4.723214149475098], dtype=object),
 array(['2007-08-28', 4.529285907745361], dtype=object),
 array(['2007-08-29', 4.788570880889893], dtype=object),
 array(['2007-08-30', 4.866071224212647], dtype=object),
 array(['2007-08-31', 4.945713996887207], dtype=object)]

In [100]:
testData = TrendDuration(closePrices)

In [117]:
GeometricCramerVonMisesTest(testData)

0.05405432215652909

In [150]:
GeometricCramerVonMisesPValue(testData)

0.7396563164034541